In [15]:
!pip install -q python-dotenv python-docx PyPDF2 langchain langchain-groq groq pydantic langchain-community sentence-transformers faiss-cpu fastapi uvicorn langchain_cohere chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 12.1 MB/s e

In [4]:
### LLMs
import os
from dotenv import load_dotenv

# Load environment variables from '.env' file
load_dotenv()

os.environ['GROQ_API_KEY'] = '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'# For LLM -- llama-3.1-8b (small) & mixtral-8x7b-32768 (large)
# os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY') # For embedding

In [5]:
ls

parser.py  pydantic_models_prompts.py  sample_data/  utils.py


In [10]:
### Build Index
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# Set embeddings
# embedding_model = CohereEmbeddings(model="embed-english-v3.0")

EMBEDDING_MODEL_NAME = "thenlper/gte-large"

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda", "trust_remote_code": True},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

<ipython-input-10-4c7224815d84>:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## Embedding jobs description

In [11]:
csv_file_path = fr"/content/job_details_all_processed.csv"

# Function to clean text by replacing newlines with spaces
def clean_text(text):
    if text is None:
        return ""
    return text.replace('\n', ' ').strip()

# Load CSV file directly as pandas DataFrame
import pandas as pd
df = pd.read_csv(csv_file_path)

# Create a document for each row
docs_list = []
for index, row in df.iterrows():
    # Create metadata from DataFrame columns
    metadata = {
        'source': csv_file_path,
        'row': index,
        'field': clean_text(str(row.get('Field', ''))),
        'experience': clean_text(str(row.get('Experience', ''))),
        'location': clean_text(str(row.get('Location', ''))),
        'company_size': clean_text(str(row.get('Company Size', ''))),
        'salary': clean_text(str(row.get('Salary', ''))),
        'job_requirements': clean_text(str(row.get('Job Requirements', ''))),
        'url': clean_text(str(row.get('URL', ''))),
    }

    # Extract additional numeric fields if they exist
    if 'Experience_year' in row:
        metadata['experience_year'] = str(row['Experience_year'])
    if 'minSalary' in row:
        metadata['min_salary'] = str(row['minSalary'])
    if 'maxSalary' in row:
        metadata['max_salary'] = str(row['maxSalary'])

    # Create page content with only Field, Location, and Job Requirements
    page_content = f"Field: {metadata['field']} Location: {metadata['location']} Job Requirements: {metadata['job_requirements']}"

    # Create document with clean metadata
    new_doc = Document(
        page_content=clean_text(page_content),
        metadata=metadata
    )
    docs_list.append(new_doc)

In [17]:
docs_list[0:3]

[Document(metadata={'source': '/content/job_details_all_processed.csv', 'row': 0, 'field': 'Xây dựng', 'experience': 'Trên 5 năm', 'location': 'Hải Phòng', 'company_size': '500-1000 nhân viên', 'salary': 'Tới 25 triệu', 'job_requirements': 'Tốt nghiệp đại học chính quy chuyên ngành kế toán, tài chính. Có chứng chỉ Kế toán trưởng. Tối thiểu 5 năm kinh nghiệm ở vị trí tương đương trong lĩnh vực xây dựng, XNK, thương mại. Am hiểu luật kế toán, thuế và chế độ tài chính doanh nghiệp. Thành thạo phần mềm kế toán (MISA), Excel tốt. Kỹ năng lãnh đạo, phân tích và xử lý vấn đề tốt. Trung thực, cẩn trọng, chịu được áp lực.', 'url': 'https://www.topcv.vn/viec-lam/ke-toan-truong-chief-accountant-mang-xay-dung-muc-luong-18-25-trieu/1744592.html?ta_source=JobSearchList_LinkDetail&u_sr_id=ix6KrGv0m4AxCKezJkohqMgppQJFdqf33kEk1SY8_1749287024', 'experience_year': '5', 'min_salary': '12.5', 'max_salary': '25.0'}, page_content='Field: Xây dựng Location: Hải Phòng Job Requirements: Tốt nghiệp đại học chính

In [16]:
vectorstore = Chroma.from_documents(
    documents=docs_list,
    collection_name="job_search",
    embedding=embedding_model,
)

In [19]:
# Create a retriever for search
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 5},  # Return top 5 most relevant job listings
)

## CV Parser and retrieval

In [21]:
# Import necessary libraries
import sys
import os
import json
from dotenv import load_dotenv
from IPython.display import display, JSON

# Make sure the GROQ_API_KEY is set
load_dotenv()  # This loads environment variables from .env file

if not os.environ.get("GROQ_API_KEY"):
    # Set your GROQ API key here if not in .env file
    os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"  # Replace with your actual API key

# Import the ResumeManager class from parser.py
from parser import ResumeManager

# Path to your resume file (PDF or DOCX)
resume_path = "htn_CV.pdf"  # Replace with your actual resume file path

# Choose your model (default is 'deepseek-r1-distill-llama-70b')
model_name = "deepseek-r1-distill-llama-70b"  # You can change this if desired



# Save the output to a JSON file
# resume_name = os.path.splitext(os.path.basename(resume_path))[0]
# output_file_path = f"{resume_name}_output.json"
# with open(output_file_path, 'w') as file:
#     json.dump(resume_manager.output, file, indent=2)

# print(f"\nOutput saved to {output_file_path}")

In [22]:
# Create the ResumeManager instance
resume_manager = ResumeManager(resume_path, model_name)

# Process the file
print("Processing resume...")
resume_manager.process_file()

# Display the results
print("\nExtracted Information:")
display(JSON(resume_manager.output))

Processing resume...


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


INFO:root:# Basic Info Extraction took 1.5802075862884521 seconds


# Basic Info Extraction took 1.5802075862884521 seconds


...

/content/parser.py:59: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>. Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.<https://python.langchain.com/docs/use_cases/extraction/>. with_structured_output does not currently support a list of pydantic schemas. If this is a blocker or if you notice other issues, please provide feedback here:<https://github.com/langchain-ai/langchain/discussions/18154>
  chain = create_extraction_chain_pydantic(target, self.model)
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


INFO:root:# Education Extraction took 1.5131337642669678 seconds


# Education Extraction took 1.5131337642669678 seconds

Extracted Information:


<IPython.core.display.JSON object>

In [23]:
bio = resume_manager.output["bio"]
skills = resume_manager.output["skills"]
education = resume_manager.output["education"]
job_title = resume_manager.output["job_title"]


for edu in education:
    qualification = edu["qualification"]
    establishment = edu["establishment"]

name = resume_manager.output["candidate_name"]
job_title = resume_manager.output["job_title"]
print(f"\nCandidate: {name}, Qualification: {qualification}")


Candidate: Huynh Trong Nghia, Qualification: Bachelor of Science in Artificial Intelligence


In [24]:
location = "Ho Chi Minh"

In [25]:
question = f"""
Bạn là một chuyên gia trong lĩnh vực tìm kiếm việc làm, bạn chuyên so sánh những thông tin được cung cấp để tìm ra những công việc chuẩn xác nhất.
Bạn sẽ được cung cấp trong CV những thông tin như tên ngành nghề: {job_title}, các kĩ năng được cung cấp bởi người dùng: {skills}, bằng cấp: {qualification}, và vị trí làm việc: {location}.

Bạn hãy tìm những công việc nào mà bạn cho là phù hợp nhất, ưu tiên những công việc có tên giống với tên ngành nghề được cung cấp trong CV, lưu ý dịch sang tiếng Anh và tiếng Việt rồi tìm kiếm để cho ra những kết quả tốt nhất,
tương tự cũng làm cho kĩ năng, bằng cấp, và vị trí làm việc. Nếu không tìm ra được việc làm phù hợp, hãy trả lời "Tôi không biết", không nên trả lời lung tung."""
print(question)


Bạn là một chuyên gia trong lĩnh vực tìm kiếm việc làm, bạn chuyên so sánh những thông tin được cung cấp để tìm ra những công việc chuẩn xác nhất.
Bạn sẽ được cung cấp trong CV những thông tin như tên ngành nghề: AI Engineer, các kĩ năng được cung cấp bởi người dùng: ['Python', 'C/C++', 'C#', 'Langchain', 'Langgraph', 'Transformers', 'Pytorch', 'Tensorflow', 'Prompt Engineering', 'LLM', 'RAG techniques', 'NLP', 'Computer Vision', 'YOLO', 'MobileNet'], bằng cấp: Bachelor of Science in Artificial Intelligence, và vị trí làm việc: Ho Chi Minh.

Bạn hãy tìm những công việc nào mà bạn cho là phù hợp nhất, ưu tiên những công việc có tên giống với tên ngành nghề được cung cấp trong CV, lưu ý dịch sang tiếng Anh và tiếng Việt rồi tìm kiếm để cho ra những kết quả tốt nhất,
tương tự cũng làm cho kĩ năng, bằng cấp, và vị trí làm việc. Nếu không tìm ra được việc làm phù hợp, hãy trả lời "Tôi không biết", không nên trả lời lung tung.


In [26]:
docs = retriever.invoke(question)

INFO:sentence_transformers.SentenceTransformer:Start multi-process pool on devices: cuda:0


Start multi-process pool on devices: cuda:0


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
for i in range (5):
    print(f"Title: {docs[i].metadata['field']}\n")

Title: Giải trí

Title: Viễn thông

Title: Viễn thông

Title: Ngân hàng

Title: Tư vấn



In [29]:
## Chatbot API Implementation

from groq import Groq
import re
import json
from typing import Dict, List, Any, Tuple

# Initialize Groq client
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

class JobSearchChatbot:
    def __init__(self, retriever, dataframe):
        self.retriever = retriever
        self.df = dataframe
        self.groq_client = groq_client

    def classify_intent(self, message: str) -> str:
        """
        Phân loại ý định của tin nhắn: 'job_search' hoặc 'chitchat'
        """
        classification_prompt = f"""
        Phân loại tin nhắn sau đây thuộc một trong hai loại:
        1. "job_search" - Nếu tin nhắn liên quan đến tìm việc làm, công việc, nghề nghiệp, kỹ năng làm việc
        2. "chitchat" - Nếu tin nhắn là chào hỏi, câu hỏi thông thường, không liên quan đến việc làm

        Tin nhắn: "{message}"

        Chỉ trả lời "job_search" hoặc "chitchat", không giải thích thêm.
        """

        try:
            response = self.groq_client.chat.completions.create(
                model="llama-3.1-8b-instant",
                messages=[{"role": "user", "content": classification_prompt}],
                temperature=0.1,
                max_tokens=10
            )

            intent = response.choices[0].message.content.strip().lower()
            return "job_search" if "job_search" in intent else "chitchat"

        except Exception as e:
            print(f"Error in intent classification: {e}")
            # Default fallback: nếu có từ khóa liên quan việc làm thì coi là job_search
            job_keywords = ['việc', 'làm', 'công việc', 'tuyển dụng', 'tìm', 'job', 'work', 'career', 'position']
            return "job_search" if any(keyword in message.lower() for keyword in job_keywords) else "chitchat"

    def handle_chitchat(self, message: str) -> Dict[str, Any]:
        """
        Xử lý tin nhắn chitchat - trả lời trực tiếp không cần truy cập CSV
        """
        chitchat_prompt = f"""
        Bạn là một trợ lý AI thân thiện. Hãy trả lời tin nhắn sau một cách tự nhiên và hữu ích:

        Tin nhắn: "{message}"

        Trả lời bằng tiếng Việt, ngắn gọn và thân thiện.
        """

        try:
            response = self.groq_client.chat.completions.create(
                model="llama-3.1-8b-instant",
                messages=[{"role": "user", "content": chitchat_prompt}],
                temperature=0.7,
                max_tokens=200
            )

            return {
                "type": "chitchat",
                "message": response.choices[0].message.content.strip(),
                "jobs": []
            }

        except Exception as e:
            return {
                "type": "chitchat",
                "message": "Xin lỗi, tôi gặp sự cố khi xử lý tin nhắn của bạn. Vui lòng thử lại.",
                "jobs": []
            }

    def extract_job_requirements(self, message: str) -> str:
        """
        Trích xuất yêu cầu tìm việc từ tin nhắn
        """
        extraction_prompt = f"""
        Từ tin nhắn sau, hãy trích xuất thông tin tìm việc bao gồm:
        - Vị trí/ngành nghề mong muốn
        - Kỹ năng có sẵn
        - Kinh nghiệm
        - Địa điểm làm việc mong muốn
        - Các yêu cầu khác

        Tin nhắn: "{message}"

        Tóm tắt thành một câu tìm kiếm rõ ràng để tìm việc làm phù hợp.
        """

        try:
            response = self.groq_client.chat.completions.create(
                model="llama-3.1-8b-instant",
                messages=[{"role": "user", "content": extraction_prompt}],
                temperature=0.3,
                max_tokens=150
            )

            return response.choices[0].message.content.strip()

        except Exception as e:
            print(f"Error extracting job requirements: {e}")
            return message  # Fallback to original message

    def search_jobs(self, query: str, max_results: int = 5) -> List[Dict[str, Any]]:
        """
        Tìm kiếm việc làm dựa trên query và trả về với index + 1
        """
        try:
            # Sử dụng retriever để tìm kiếm
            docs = self.retriever.invoke(query)

            jobs = []
            for i, doc in enumerate(docs[:max_results]):
                # Lấy row index từ metadata
                row_index = doc.metadata.get('row', i)

                # Tạo job object với index + 1 để tương thích với frontend
                job = {
                    "id": row_index + 1,  # index + 1 để match với job_details.html
                    "title": doc.metadata.get('field', 'N/A'),
                    "company": doc.metadata.get('company_size', 'N/A'),
                    "location": doc.metadata.get('location', 'N/A'),
                    "experience": doc.metadata.get('experience', 'N/A'),
                    "salary": doc.metadata.get('salary', 'N/A'),
                    "requirements": doc.metadata.get('job_requirements', 'N/A')[:200] + "..." if len(doc.metadata.get('job_requirements', '')) > 200 else doc.metadata.get('job_requirements', 'N/A'),
                    "url": doc.metadata.get('url', '#')
                }
                jobs.append(job)

            return jobs

        except Exception as e:
            print(f"Error in job search: {e}")
            return []

    def handle_job_search(self, message: str) -> Dict[str, Any]:
        """
        Xử lý tin nhắn tìm việc - truy cập CSV và trả về danh sách việc làm
        """
        # Trích xuất yêu cầu tìm việc
        search_query = self.extract_job_requirements(message)

        # Tìm kiếm việc làm
        jobs = self.search_jobs(search_query, max_results=5)

        # Tạo phản hồi
        if jobs:
            job_list = "\n".join([
                f"{i+1}. {job['title']} tại {job['location']} - {job['salary']}"
                for i, job in enumerate(jobs)
            ])

            response_message = f"""Tôi đã tìm thấy {len(jobs)} công việc phù hợp với yêu cầu của bạn:

{job_list}

Bạn có thể click vào từng công việc để xem chi tiết. Có cần tôi tìm thêm thông tin gì khác không?"""
        else:
            response_message = "Xin lỗi, tôi không tìm thấy công việc nào phù hợp với yêu cầu của bạn. Bạn có thể thử với từ khóa khác không?"

        return {
            "type": "job_search",
            "message": response_message,
            "jobs": jobs
        }

    def process_message(self, message: str) -> Dict[str, Any]:
        """
        Xử lý tin nhắn chính - phân loại và gọi handler tương ứng
        """
        # Phân loại ý định
        intent = self.classify_intent(message)

        # Xử lý theo ý định
        if intent == "job_search":
            return self.handle_job_search(message)
        else:
            return self.handle_chitchat(message)


In [30]:
# Khởi tạo chatbot với retriever và dataframe đã có
chatbot = JobSearchChatbot(retriever, df)

print("Chatbot đã được khởi tạo thành công!")
print("Sẵn sàng xử lý tin nhắn với 2 luồng: chitchat và job_search")


Chatbot đã được khởi tạo thành công!
Sẵn sàng xử lý tin nhắn với 2 luồng: chitchat và job_search


In [31]:
## Test Chatbot API

def chat_api(message: str) -> Dict[str, Any]:
    """
    API endpoint để xử lý tin nhắn từ frontend
    Trả về format JSON tương thích với job_listing.html
    """
    try:
        result = chatbot.process_message(message)

        # Thêm thông tin debug
        print(f"Input: {message}")
        print(f"Type: {result['type']}")
        print(f"Jobs found: {len(result.get('jobs', []))}")

        return {
            "success": True,
            "data": result
        }

    except Exception as e:
        print(f"Error in chat_api: {e}")
        return {
            "success": False,
            "error": str(e),
            "data": {
                "type": "error",
                "message": "Xin lỗi, có lỗi xảy ra. Vui lòng thử lại.",
                "jobs": []
            }
        }

# Test với các loại tin nhắn khác nhau
test_messages = [
    "Xin chào, bạn có khỏe không?",  # chitchat
    "Tôi đang tìm việc làm về AI ở TP.HCM",  # job_search
    "Cảm ơn bạn!",  # chitchat
    "Tìm công việc Python developer có kinh nghiệm 2 năm"  # job_search
]

print("=== TESTING CHATBOT API ===\n")

for i, message in enumerate(test_messages, 1):
    print(f"Test {i}: {message}")
    result = chat_api(message)
    print(f"Response: {result['data']['message']}")

    if result['data']['jobs']:
        print(f"Jobs returned: {len(result['data']['jobs'])}")
        for job in result['data']['jobs'][:2]:  # Show first 2 jobs
            print(f"  - Job ID {job['id']}: {job['title']} at {job['location']}")

    print("-" * 50)


=== TESTING CHATBOT API ===

Test 1: Xin chào, bạn có khỏe không?


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Input: Xin chào, bạn có khỏe không?
Type: chitchat
Jobs found: 0
Response: Xin chào! Cảm ơn bạn đã hỏi. Tôi đang khỏe mạnh và sẵn sàng giúp đỡ bạn! Bạn cần hỗ trợ gì hôm nay?
--------------------------------------------------
Test 2: Tôi đang tìm việc làm về AI ở TP.HCM


...

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


INFO:sentence_transformers.SentenceTransformer:Start multi-process pool on devices: cuda:0


Start multi-process pool on devices: cuda:0


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Input: Tìm công việc Python developer có kinh nghiệm 2 năm
Type: job_search
Jobs found: 5
Response: Tôi đã tìm thấy 5 công việc phù hợp với yêu cầu của bạn:

1. Giáo dục / Đào tạo tại Hà Nội - Thoả thuận
2. Viễn thông tại Hà Nội - Thoả thuận
3. Viễn thông tại Hà Nội - Thoả thuận
4. Sản xuất tại Hà Nội - 20 - 23 triệu
5. IT - Phần mềm tại Hà Nội - 15 - 20 triệu

Bạn có thể click vào từng công việc để xem chi tiết. Có cần tôi tìm thêm thông tin gì khác không?
Jobs returned: 5
  - Job ID 251: Giáo dục / Đào tạo at Hà Nội
  - Job ID 149: Viễn thông at Hà Nội
--------------------------------------------------


In [32]:
## Frontend Integration Functions

def format_jobs_for_frontend(jobs: List[Dict[str, Any]]) -> str:
    """
    Format danh sách việc làm để tích hợp với job_listing.html
    Tạo HTML links clickable với job_details.html
    """
    if not jobs:
        return ""

    html_content = "<div class='job-results'>"

    for job in jobs:
        job_url = f"job_details.html?id={job['id']}"
        html_content += f"""
        <div class='job-item' data-job-id='{job['id']}'>
            <h4><a href='{job_url}' class='job-link'>{job['title']}</a></h4>
            <p><strong>Địa điểm:</strong> {job['location']}</p>
            <p><strong>Kinh nghiệm:</strong> {job['experience']}</p>
            <p><strong>Lương:</strong> {job['salary']}</p>
            <p><strong>Mô tả:</strong> {job['requirements']}</p>
            <a href='{job_url}' class='btn btn-primary'>Xem chi tiết</a>
        </div>
        <hr>
        """

    html_content += "</div>"
    return html_content

def generate_javascript_integration() -> str:
    """
    Tạo JavaScript code để tích hợp với frontend
    """
    js_code = """
    // JavaScript integration for chatbot API
    function sendChatMessage(message) {
        // Gọi API chatbot (cần implement endpoint thực tế)
        fetch('/api/chat', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({message: message})
        })
        .then(response => response.json())
        .then(data => {
            if (data.success) {
                displayChatResponse(data.data);
            } else {
                console.error('Chat API error:', data.error);
            }
        })
        .catch(error => {
            console.error('Network error:', error);
        });
    }

    function displayChatResponse(responseData) {
        const chatContainer = document.getElementById('chat-messages');

        // Hiển thị tin nhắn phản hồi
        const messageElement = document.createElement('div');
        messageElement.className = 'chat-message bot-message';
        messageElement.textContent = responseData.message;
        chatContainer.appendChild(messageElement);

        // Nếu có kết quả việc làm, hiển thị dưới dạng clickable links
        if (responseData.jobs && responseData.jobs.length > 0) {
            const jobsContainer = document.createElement('div');
            jobsContainer.className = 'jobs-container';

            responseData.jobs.forEach(job => {
                const jobElement = document.createElement('div');
                jobElement.className = 'job-item-chat';
                jobElement.innerHTML = `
                    <h5><a href="job_details.html?id=${job.id}" target="_blank">${job.title}</a></h5>
                    <p>${job.location} - ${job.salary}</p>
                `;
                jobsContainer.appendChild(jobElement);
            });

            chatContainer.appendChild(jobsContainer);
        }

        // Scroll to bottom
        chatContainer.scrollTop = chatContainer.scrollHeight;
    }
    """
    return js_code

# In ra integration code
print("=== FRONTEND INTEGRATION ===")
print("\n1. HTML Jobs Format Example:")
example_jobs = [
    {
        "id": 123,
        "title": "AI Engineer",
        "location": "TP.HCM",
        "experience": "2 năm",
        "salary": "20-30 triệu",
        "requirements": "Có kinh nghiệm Python, Machine Learning..."
    }
]

print(format_jobs_for_frontend(example_jobs))

print("\n2. JavaScript Integration Code:")
print("Sử dụng code này trong job_listing.html để tích hợp chatbot:")
print(generate_javascript_integration())

print("\n=== READY FOR PRODUCTION ===")
print("Chatbot API đã sẵn sàng!")
print("- chat_api(message) -> trả về JSON với jobs có id (index + 1)")
print("- Jobs có thể click để mở job_details.html?id={job_id}")
print("- Phân loại tự động chitchat vs job_search")
print("- Không truy cập CSV cho chitchat, chỉ cho job_search")


=== FRONTEND INTEGRATION ===

1. HTML Jobs Format Example:
<div class='job-results'>
        <div class='job-item' data-job-id='123'>
            <h4><a href='job_details.html?id=123' class='job-link'>AI Engineer</a></h4>
            <p><strong>Địa điểm:</strong> TP.HCM</p>
            <p><strong>Kinh nghiệm:</strong> 2 năm</p>
            <p><strong>Lương:</strong> 20-30 triệu</p>
            <p><strong>Mô tả:</strong> Có kinh nghiệm Python, Machine Learning...</p>
            <a href='job_details.html?id=123' class='btn btn-primary'>Xem chi tiết</a>
        </div>
        <hr>
        </div>

2. JavaScript Integration Code:
Sử dụng code này trong job_listing.html để tích hợp chatbot:

    // JavaScript integration for chatbot API
    function sendChatMessage(message) {
        // Gọi API chatbot (cần implement endpoint thực tế)
        fetch('/api/chat', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            